# Попытаемся что-то сделать, собственно #

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()

In [2]:
import pathlib

DATA_DIR = pathlib.Path("../data/")
train_target = pd.read_csv(DATA_DIR.joinpath("pet_target_train.csv"), index_col="date", parse_dates=["date"])
test_target = pd.read_csv(DATA_DIR.joinpath("pet_test_timestamps.csv"), index_col="date", parse_dates=["date"])
daily = pd.read_csv(DATA_DIR.joinpath("pet_daily.csv"), index_col="date", parse_dates=["date"])
weekly = pd.read_csv(DATA_DIR.joinpath("pet_weekly.csv"), index_col="date", parse_dates=["date"])

## Feature generation ##

Так, ну это код из бейзлайна, тут ничего лучше не придумать - мы делаем признаки на каждый день.

In [3]:
dfts = daily.resample("D").mean()
wfts = weekly.resample("D").ffill()
fts = dfts.join(wfts)

Опять же, код из бейзлайна.

In [33]:
PERIODS = ["1D", "7D", "30D", "60D", "180D"]
AGGREGATES = ["mean", "median", "std", "max", "min"]

all_features = []

for period in PERIODS:
    for agg in AGGREGATES:
        if agg!="mean" and period=="1D":
            continue
        rolling_features = dfts.rolling(period).aggregate(agg)
        rolling_features.rename(lambda x: "_".join([x, period, agg]), axis=1, inplace=True)
        all_features.append(rolling_features[rolling_features.index.day==9])

In [34]:
all_features = pd.concat(all_features, axis=1)

all_features["prediction_date"] = (all_features.index
                                   + pd.TimedeltaIndex(all_features.index.days_in_month-8,
                                                       unit="D"))
final_fts = all_features.ffill()

In [35]:
data = train_target.join(final_fts.set_index("prediction_date"), how="outer")["2004-06":]
data.index.name = "date"
data.head()

,pet,brent_close_1D_mean,brent_open_1D_mean,brent_max_1D_mean,brent_min_1D_mean,USDCNY_close_1D_mean,USDCNY_open_1D_mean,USDCNY_max_1D_mean,USDCNY_min_1D_mean,brent_close_7D_mean,...,USDCNY_max_180D_max,USDCNY_min_180D_max,brent_close_180D_min,brent_open_180D_min,brent_max_180D_min,brent_min_180D_min,USDCNY_close_180D_min,USDCNY_open_180D_min,USDCNY_max_180D_min,USDCNY_min_180D_min
date,,,,,,,,,,,,,,,,,,,,,
2004-06-01,961.25,32.23,32.95,33.14,32.15,8.2767,8.2767,8.2767,8.2767,36.5450,...,8.2774,8.2774,27.86,28.05,28.43,27.68,8.2767,8.2767,8.2767,8.2767
2004-07-01,962.00,35.29,34.75,35.44,34.43,8.2767,8.2767,8.2767,8.2767,35.6740,...,8.2774,8.2774,28.83,28.94,29.18,28.44,8.2767,8.2767,8.2767,8.2767
2004-08-01,1053.75,37.05,37.60,37.80,37.00,8.2769,8.2769,8.2769,8.2769,36.9820,...,8.2774,8.2774,28.83,29.00,29.18,28.44,8.2766,8.2766,8.2766,8.2766
2004-09-01,1180.00,37.05,37.60,37.80,37.00,8.2769,8.2769,8.2769,8.2769,40.5225,...,8.2774,8.2774,30.07,29.65,30.15,29.57,8.2766,8.2766,8.2766,8.2766
2004-10-01,1240.00,42.22,40.25,42.30,40.10,8.2768,8.2768,8.2768,8.2768,41.0440,...,8.2774,8.2774,30.21,30.69,30.92,29.95,8.2766,8.2766,8.2766,8.2766


Итак, теперь у нас есть датафрейм, на котором нам надо просто сделать регрессию. С ним особенно ничего не сделаешь, так что попробуем нагенерить всякой дичи.

In [38]:
df = data.copy()
cols = list(data.columns)
cols.remove('pet')
for i, col1 in enumerate(cols):
    for col2 in cols[i + 1:]:
        new_col = data[col1] * data[col2]
        df[col1 + '__' + col2] = new_col
df.head()

,pet,brent_close_1D_mean,brent_open_1D_mean,brent_max_1D_mean,brent_min_1D_mean,USDCNY_close_1D_mean,USDCNY_open_1D_mean,USDCNY_max_1D_mean,USDCNY_min_1D_mean,brent_close_7D_mean,...,brent_min_180D_min__USDCNY_close_180D_min,brent_min_180D_min__USDCNY_open_180D_min,brent_min_180D_min__USDCNY_max_180D_min,brent_min_180D_min__USDCNY_min_180D_min,USDCNY_close_180D_min__USDCNY_open_180D_min,USDCNY_close_180D_min__USDCNY_max_180D_min,USDCNY_close_180D_min__USDCNY_min_180D_min,USDCNY_open_180D_min__USDCNY_max_180D_min,USDCNY_open_180D_min__USDCNY_min_180D_min,USDCNY_max_180D_min__USDCNY_min_180D_min
date,,,,,,,,,,,,,,,,,,,,,
2004-06-01,961.25,32.23,32.95,33.14,32.15,8.2767,8.2767,8.2767,8.2767,36.5450,...,229.099056,229.099056,229.099056,229.099056,68.503763,68.503763,68.503763,68.503763,68.503763,68.503763
2004-07-01,962.00,35.29,34.75,35.44,34.43,8.2767,8.2767,8.2767,8.2767,35.6740,...,235.389348,235.389348,235.389348,235.389348,68.503763,68.503763,68.503763,68.503763,68.503763,68.503763
2004-08-01,1053.75,37.05,37.60,37.80,37.00,8.2769,8.2769,8.2769,8.2769,36.9820,...,235.386504,235.386504,235.386504,235.386504,68.502108,68.502108,68.502108,68.502108,68.502108,68.502108
2004-09-01,1180.00,37.05,37.60,37.80,37.00,8.2769,8.2769,8.2769,8.2769,40.5225,...,244.739062,244.739062,244.739062,244.739062,68.502108,68.502108,68.502108,68.502108,68.502108,68.502108
2004-10-01,1240.00,42.22,40.25,42.30,40.10,8.2768,8.2768,8.2768,8.2768,41.0440,...,247.884170,247.884170,247.884170,247.884170,68.502108,68.502108,68.502108,68.502108,68.502108,68.502108


In [40]:
train = df[:'2015-12-01']

Теперь найдем корреляции всей этой фигни с таргетом.

In [74]:
from scipy.stats import pearsonr

corrs = {}
for col in train.columns:
    corrs[col] = abs(pearsonr(train[col], train_target['pet'])[0])

In [75]:
corrs = list(zip(corrs.values(), corrs.keys()))
corrs.sort(reverse=True)

In [76]:
corrs[:10]

[(1.0, 'pet'),
 (0.768275266657408, 'brent_close_7D_min__USDCNY_open_180D_min'),
 (0.768215603718349, 'brent_close_7D_min__USDCNY_min_180D_min'),
 (0.7682122013718794, 'brent_close_7D_min__USDCNY_max_180D_min'),
 (0.7681783423797799, 'brent_close_7D_min__USDCNY_close_180D_min'),
 (0.7681778263627166, 'brent_close_7D_min__USDCNY_open_180D_median'),
 (0.7681482608896524, 'brent_close_7D_min__USDCNY_max_180D_median'),
 (0.7681244776707767, 'brent_close_7D_min__USDCNY_min_180D_median'),
 (0.7681168558723362, 'brent_close_7D_min__USDCNY_close_180D_median'),
 (0.767567623659583, 'brent_min_7D_min__USDCNY_open_180D_min')]

Заметим, что корреляции стали больше!

In [79]:
corrs.remove((1., 'pet'))
final_cols = [el[1] for el in corrs[:200]]

In [82]:
train = df[final_cols][:'2013']
val = df[final_cols]['2014':'2015']
test = df[final_cols]['2016':]

Как-то так.

## Моделька ##

In [88]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

In [87]:
from sklearn.svm import SVR
model = SVR()
model.fit(train.values, train_target[:'2013'].values)

/home/data_sapiens/Anaconda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/data_sapiens/Anaconda/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
    gamma='auto_deprecated', kernel='rbf', max_iter=-1, shrinking=True,
    tol=0.001, verbose=False)

In [89]:
train_pred = model.predict(train.values)
val_pred = model.predict(val.values)
test_pred = model.predict(test.values)

In [91]:
print(f"TR performance: {mape(train_target[:'2013'].values, train_pred)}")
print(f"CV performance: {mape(train_target['2014':'2015'].values, val_pred)}")

TR performance: 0.1266195222403239
CV performance: 0.2094885023912846


In [106]:
test_target['pet'] = test_pred
test_target.to_csv('../data/noidea.csv')

Короче, это нифига не работает, и ладно.